In [12]:
import os
import sys
import urllib.request
import datetime
import time
import json
client_id=
client_secret=


In [13]:
#[CODE 1]
# url 을 받으면
def getRequestUrl(url):
    # 인증 코드들을 url 에 더해 request 를 만듬
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    try:
        # 그 request 를 진행
        response = urllib.request.urlopen(req)
        # ok 응답이 오면
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            # 응답을 utf-8로 읽어서 반환
            return response.read().decode('utf-8')
    # 에러 발생시 에러 출력
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [14]:
 #[CODE 2]

def getNaverSearch(node, srcText, start, display):
    # node 와 base를 결합해 naver api 주소 만듬, node 는 blog, news 등등
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    # queryString 만들기 검색어는 urllib.parse.quote(텍스트(입력받은 String))) 나머지는 api 가 원하는 요구사항
    parameters = "?query=%s&start=%s&display=%s" %(urllib.parse.quote(srcText), start, display)
    # 합쳐서 url 만들기
    url = base + node + parameters
    # 이 url 을 호출 그리고 반환을 보고 없으면 넘김, 있으면 json 형태니까(api 호출로 받아온, 지정하면 xml) Python 객체로 반환
    responseDecode = getRequestUrl(url) #[CODE 1]
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

In [15]:
#[CODE 3]
def getPostData(post, jsonResult, cnt):
    # post(item) 에서 데이터 추출할당
    title = post['title']
    description = post['description']
    link = post['link']
    # 시간 데이터를 python 에 맞게 변환
    pDate = datetime.datetime.strptime(post['postdate'], '%Y%m%d')
    pDate = pDate.strftime('%Y-%m-%d')
    # 결과에 더함
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 'link': link, 'pDate':pDate})
    return

In [16]:
#[CODE 0]
def main():
    # 함수에 넣을 변수 정의
    node = 'blog' #크롤링할 대상
    srcText = input('검색어를 입력하세요: ')
    cnt = 0
    jsonResult = []
    # 바로 api 호출, 그리고 총 개수 할당
    jsonResponse = getNaverSearch(node, srcText, 1, 100) #[CODE 2] 여기서 반환받는값은 json 이 아니라 python 객체
    total = jsonResponse['total'] # 그래서 슬라이싱을 쓸 수 있음
    # 호출의 결과가 존재하는 동안
    while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
        # 결과 데이터를 각각 jsonResult 에 더함
        for post in jsonResponse['items']:
            cnt += 1
            getPostData(post, jsonResult, cnt) #[CODE 3]
        # start~display 값은 불러왔으니 다음, start+display 부터 display 만큼 데이터 불러옴 이것을 호출의 결과가 존재하는 동안, 즉 끝까지 반복
        if(cnt == 1000):
            break
        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100) #[CODE 2]
    print('전체 검색 : %d 건' %total)

    # 결과를 json 파일로 저장
    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult, indent = 4, sort_keys = True, ensure_ascii = False)
        outfile.write(jsonFile)
    print("가져온 데이터 : %d 건" %(cnt))
    print('%s_naver_%s.json SAVED' % (srcText, node))

if __name__ == '__main__':
    main()

[2025-02-19 12:10:20.320064] Url Request Success
[2025-02-19 12:10:20.622659] Url Request Success
[2025-02-19 12:10:20.921490] Url Request Success
[2025-02-19 12:10:21.227465] Url Request Success
[2025-02-19 12:10:21.581971] Url Request Success
[2025-02-19 12:10:21.916839] Url Request Success
[2025-02-19 12:10:22.229538] Url Request Success
[2025-02-19 12:10:22.559539] Url Request Success
[2025-02-19 12:10:22.894376] Url Request Success
[2025-02-19 12:10:23.239154] Url Request Success
전체 검색 : 7860086 건
가져온 데이터 : 1000 건
신상_naver_blog.json SAVED
